In [1]:
line_dict = {}
line_noun_dict = {}
import nltk
import re
import spacy
from spacy import *
nlp=spacy.load('en')
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.corpus import stopwords
import xml.dom.minidom as minidom
#doc = minidom.parse('DevelopmentData_QuestionsFromSentences.xml')
#itemlist = doc.getElementsByTagName('text') 
itemlist=['John eat mango at 9pm.','Rabbit eat carrot near the Ganges river.','John plays football.','Harry studies in University_of_California, Los Angeles.','Pierre teaches Jean','Narender Modi is Prime Minister of India']
itemlist.append('Barack Obama is President of the US')
result=''
res = []
# from nltk.tokenize import RegexpTokenizer

# tokenizer = RegexpTokenizer(r'\w+')
j = 0
noun_chunk = {}
FullNameEntityDict = {}
FullSubjectObjectDict = {}
subject_dict = {}
lis = []
for i in itemlist:
#     memoryElem = i
    j=j+1
#     result=''.join( [node.data for node in memoryElem.childNodes] )
#     result=itemlist[i]
#     print("result",result)
    res = nlp(i)
#     print("res",res)
    line_dict[j] = result
    tempDict={}
    for np in res.ents:
        tempDict.update({str(np):np.label_})

    FullNameEntityDict[j] = tempDict
    
    #get token dependencies
    so_obj_Dict={}
    subjects=[]
    objects=[]
    verb_tag_Dict={} #stores words with verb tag
    for text in res:
        print(text,text.dep_)
        if text.dep_ == "nsubj" or text.dep_=='npadvmod' or text.dep_=='compound':
            #subject would be
            subject = text.orth_
            subjects.append(subject)
#             print("subject",subject)
#             lis.append(subject)
#             so_obj_Dict.update({"subject":subject})
        elif (re.match(r'.obj',text.dep_)):
            #iobj for indirect object
            any_object = text.orth_
            objects.append(any_object)
#             print("iobj",indirect_object)
#             lis.append(indirect_object)
            so_obj_Dict.update({"object":any_object})
        elif (re.match(r'VB',text.dep_)or re.match(r'ROOT',text.dep_)):
            #iobj for indirect object
            verb_tag_Dict.update({text.dep_:text})
#             print("iobj",indirect_object)
#             lis.append(indirect_object)
        
        so_obj_Dict.update({"object":objects})
        so_obj_Dict.update({"subject":subjects})
        so_obj_Dict.update({"verb":verb_tag_Dict}) 
        
    
    pp_tag_List=[]   #stores words with prepositions tag
    for word in list(res.sents)[0]:
        print(word,word.tag_)
        '''if(re.match(r'^VB*',word.tag_)):
#             print("word.tag_",word.tag_)
            verb_tag_Dict.update({word.tag_:word})'''
            #word_tag_dict[word]=word.tag_
        if(re.match(r'IN',word.tag_)):
#             print("word.tag_",word.tag_)
            pp_tag_List.append(word)
            #word_tag_dict[word]=word.tag_
        
        so_obj_Dict.update({"pp":pp_tag_List}) 
    #print(so_obj_Dict)
    FullSubjectObjectDict[j] = so_obj_Dict
    #subject_dict[j]=lis
print("FullNameEntityDict",FullNameEntityDict)
print("FullSubjectObjectDict",FullSubjectObjectDict)

# print("FullNameEntityDict",FullNameEntityDict)
# print(FullSubjectObjectDict)
#Person-Person- :Who + verb + object
for i in range (1,len(FullSubjectObjectDict)+1):
    print(i)
    tempDict=FullSubjectObjectDict.get(i)
    #print(tempDict)
    subTempList=tempDict['subject']
    objTempList=tempDict['object']
    ppTempList=tempDict['pp']
    verbTempDict=tempDict['verb']
    
    temp={}
    if(len(subTempList)!=0 and len(objTempList)!=0 and len(ppTempList)!=0):
        if(FullNameEntityDict.get(i).get(subTempList[0])=='PERSON' and FullNameEntityDict.get(i).get(objTempList[0])!='PERSON'):
            if(verbTempDict.get('ROOT')!=None):
#                 print("1")
                print('Who '+str(verbTempDict.get('ROOT'))+' '+str(objTempList[0])+'?')
                if(FullNameEntityDict.get(i).get(objTempList[0])=='GPE'):
                    print('Where '+str(subTempList[0])+' '+str(verbTempDict.get('ROOT'))+'?')
                else:
                    print('What '+str(subTempList[0])+' '+str(verbTempDict.get('ROOT'))+'?')
            if(verbTempDict.get('VB')!=None):
#                 print("2")
                print('Who '+str(verbTempDict.get('VB'))+' '+str(objTempList[0])+'?')
                print('What '+str(subTempList[0])+' '+str(verbTempDict.get('VB'))+'?')
            
                
    elif(len(subTempList)!=0 and len(objTempList)!=0 and len(ppTempList)==0):
        if(FullNameEntityDict.get(i).get(subTempList[0])=='PERSON' and FullNameEntityDict.get(i).get(objTempList[0])=='PERSON'):
            if(verbTempDict.get('ROOT')!=None):
#                 print("3")
                print('Who '+str(verbTempDict.get('ROOT'))+' '+str(objTempList[0])+'?')
                print('Whom '+str(subTempList[0])+' '+str(verbTempDict.get('ROOT'))+'?')
        elif(FullNameEntityDict.get(i).get(subTempList[0])=='PERSON' and FullNameEntityDict.get(i).get(objTempList[0])!='PERSON'):
            if(verbTempDict.get('ROOT')!=None):
#                 print("4")
                print('Who '+str(verbTempDict.get('ROOT'))+ ' '+str(objTempList[0])+'?')
                
                

John compound
eat ROOT
mango dobj
at prep
9 nummod
pm pobj
. punct
John NNP
eat VB
mango NN
at IN
9 CD
pm NN
. .
Rabbit nsubj
eat ROOT
carrot dobj
near prep
the det
Ganges compound
river pobj
. punct
Rabbit NNP
eat VB
carrot NN
near IN
the DT
Ganges NNP
river NN
. .
John nsubj
plays ROOT
football dobj
. punct
John NNP
plays VBZ
football NN
. .
Harry compound
studies ROOT
in prep
University_of_California pobj
, punct
Los compound
Angeles appos
. punct
Harry NNP
studies NNS
in IN
University_of_California NNP
, ,
Los NNP
Angeles NNP
. .
Pierre nsubj
teaches ROOT
Jean dobj
Pierre NNP
teaches VBZ
Jean NNP
Narender compound
Modi nsubj
is ROOT
Prime compound
Minister ccomp
of prep
India pobj
Narender NNP
Modi NNP
is VBZ
Prime NNP
Minister NNP
of IN
India NNP
Barack compound
Obama nsubj
is ROOT
President attr
of prep
the det
US pobj
Barack NNP
Obama NNP
is VBZ
President NNP
of IN
the DT
US NNP
FullNameEntityDict {1: {'9pm': 'TIME', 'John': 'PERSON'}, 2: {'Ganges': 'LOC', 'Rabbit': 'PERSON'}, 3